# How to find answers using jupyter?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

How to sanity check the environment?


In [6]:
// initialize NEL
try {
var deps = ['npm install', 'JSONStream', 'nel', 'path', 'fs', 'vm'];
require('child_process').execSync(deps.slice(0, 3).join(' '));
deps.slice(1).forEach(v => (global[v] = require(v)));

// How to parse cells of {types} from a notebook?
var getCells = (notebook, types = ['code']) => {
    var cells = [];
    var file = fs.createReadStream(notebook)
        .pipe(JSONStream.parse('cells.*'));
    file.on('data', (cell) => {
        if(types.indexOf(cell.cell_type) > -1) {
            cells[cells.length] = cell.source.join('').trim();
        }
    });
    // create a promise out of this stream
    return new Promise((resolve, reject) => {
        file.on('error', e => reject(e, notebook));
        file.on('close', () => resolve(cells, notebook))
    });
};

// How to run all promises sequentially?
var runAllPromises = (promises) => {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (new Promise(func))
                .then(Array.prototype.concat.bind(result));
        });
    },Promise.resolve([]));
};

// How to create VM promises out of the cell content?
var runAll = (cells) => {
    var sess = new nel.Session();
    return runAllPromises(cells.map(
        (cell) => (resolve) => sess.execute(cell, {
            onSuccess: (o) => resolve(o),
            onError: (o) => resolve(o),
        })
    ));
};

// How to test to make sure the VM works?
$$.async();
getCells('How to find answers.ipynb')
    .then(cells => runAll(cells)
          .then((r) => $$.sendResult(getCells)));
}catch(e){$$.sendError(e)}
// $$.done() // for our dumb parser


Promise { <pending> }

[Function: getCells]

How to import a {notebook}?

How to test the import feature?


In [4]:
if (typeof imported == 'undefined') { var imported = {}; }

// display a heading when new notebooks are imported
var runWithHeader = (cells, filename) => {
    console.log('Running ' + filename  
                + ' with ' + cells.length + ' cells');
    return (imported[filename] = runAll(cells));
};

// cache results when notebooks are processed
var assignResults = (results, filename) => {
    var results = results.reduce((obj, r, i) => {
        if (typeof r === 'function') {
            obj[filename+'['+i+']'] = r
        }
        return obj;
    }, {});
    Object.assign(global, results);
    imported[filename] = Promise.resolve(results);
    return results;
};

// provide a function for importing any notebook as a module and executing it
var importNotebook = (notebook) => {
    try {
    var filename = path.basename(notebook);
    if(typeof notebook === 'undefined') {
        return Promise.resolve([]);
    }
    if (typeof imported[filename] !== 'undefined') {
        console.log('Already imported');
        console.log(Object.keys(imported));
        return imported[filename];
    }
    return getCells(notebook, 'code')
        .then((cells) => runWithHeader(cells, filename))
        .then((results) => assignResults(results, filename));
    }catch(e){console.log(e); return Promise.resolve([])}
};

// use this document as the test results to prevent recursion
imported['How to find answers.ipynb'] = Promise.resolve({
    'How to find answers.ipynb[0]': getCells,
    'How to find answers.ipynb[1]': importNotebook
});

// test to make sure it works
$$.async();
importNotebook('How to find answers.ipynb')
    .then((results) => {
        $$.sendResult(importNotebook);
    }).catch((e) => $$.sendError(e));
// $$.done()


Already imported
[ 'How to find answers.ipynb' ]


[Function: importNotebook]

These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?



In [7]:
//$$.async();
//importNotebook('How to interpret questions.ipynb')
//    .then(() => $$.done('done importing importing'));

$$.async();

setTimeout(() => $$.sendResult('hit2'), 2000);
// $$.done()


'hit2'